In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub
import pickle

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        with open(f"{model_name}_model", "wb") as f:
            pickle.dump(model, f)

        with open(f"{model_name}_model", "rb") as f:
            clf = pickle.load(f)

        mlflow.log_artifact(f"{model_name}_model", "model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()

[I 2025-08-26 21:08:18,657] A new study created in memory with name: no-name-1f6bb3ce-83d0-49b4-a596-69efcf634d94
[I 2025-08-26 21:14:56,628] Trial 0 finished with value: 0.6932978723404255 and parameters: {'C': 0.14603752847012919, 'kernel': 'poly'}. Best is trial 0 with value: 0.6932978723404255.
[I 2025-08-26 21:23:25,763] Trial 1 finished with value: 0.4053191489361702 and parameters: {'C': 0.0039607531940575634, 'kernel': 'rbf'}. Best is trial 0 with value: 0.6932978723404255.
[I 2025-08-26 21:29:48,220] Trial 2 finished with value: 0.7662765957446809 and parameters: {'C': 0.2003970790104463, 'kernel': 'rbf'}. Best is trial 2 with value: 0.7662765957446809.
[I 2025-08-26 21:36:08,244] Trial 3 finished with value: 0.7410638297872341 and parameters: {'C': 0.12732966998098744, 'kernel': 'rbf'}. Best is trial 2 with value: 0.7662765957446809.
[I 2025-08-26 21:43:28,997] Trial 4 finished with value: 0.5569148936170213 and parameters: {'C': 0.0841578398191368, 'kernel': 'poly'}. Best is